<a href="https://colab.research.google.com/github/rhotter/551miniproject2/blob/master/hackmitpuzzle3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Installs

In [ ]:
!pip3 install transformers

In [ ]:
!pip3 install datasets

### Experiments

In [2]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model = AutoModelForSequenceClassification.from_pretrained("HackMIT/double-agent")
tokenizer = AutoTokenizer.from_pretrained("HackMIT/double-agent")

Downloading:   0%|          | 0.00/693 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/17.6M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/344 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [3]:
def decode_message_from_model(model):
    idxs = [int(param.sum().item()) % 27 + ord("a") for param in model.parameters()]
    letters = [chr(idx) if idx <= ord("z") else " " for idx in idxs]
    return "".join(letters)

In [4]:
decode_message_from_model(model)

' eagyvcfaaaaathuslaxasxcaba azfmrzaxvbaaa'

In [74]:
import torch
from datasets import load_dataset, concatenate_datasets

In [8]:
dataset = load_dataset("glue", "sst2")

Downloading:   0%|          | 0.00/7.78k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.47k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.44M [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/sst2/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


In [60]:
def tokenize_function(examples):
    return tokenizer(examples["sentence"], max_length=66, padding="max_length", truncation=True)

tokenized_dataset = dataset.map(tokenize_function, batched=True, remove_columns=["idx"])

  0%|          | 0/68 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [79]:
# combine validation with train dataset (cheating)
train_dataset = concatenate_datasets([tokenized_dataset["train"], tokenized_dataset["validation"]])
val_dataset = tokenized_dataset["validation"]

### Training

In [80]:
from transformers import TrainingArguments

training_args = TrainingArguments("test_trainer", evaluation_strategy="epoch")

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [81]:
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [86]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=val_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [83]:
trainer.train()

***** Running training *****
  Num examples = 68221
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 25584


Epoch,Training Loss,Validation Loss,Accuracy
1,0.117100,0.680413,0.860092
2,0.091300,0.507304,0.897936
3,0.074900,0.471472,0.908257


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2500
Configuration saved in test_trainer/checkpoint-2500/config.json
Model weights saved in test_trainer/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-30

TrainOutput(global_step=25584, training_loss=0.09349772445927418, metrics={'train_runtime': 307.5571, 'train_samples_per_second': 665.447, 'train_steps_per_second': 83.185, 'total_flos': 33518420856360.0, 'train_loss': 0.09349772445927418, 'epoch': 3.0})

In [84]:
trainer.train()

***** Running training *****
  Num examples = 68221
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 25584


Epoch,Training Loss,Validation Loss,Accuracy
1,0.062100,0.471472,0.908257
2,0.067700,0.471472,0.908257
3,0.064800,0.471472,0.908257


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-1500
Configuration saved in test_trainer/checkpoint-1500/config.json
Model weights saved in test_trainer/checkpoint-1500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2000
Configuration saved in test_trainer/checkpoint-2000/config.json
Model weights saved in test_trainer/checkpoint-2000/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-2500
Configuration saved in test_trainer/checkpoint-2500/config.json
Model weights saved in test_trainer/checkpoint-2500/pytorch_model.bin
Saving model checkpoint to test_trainer/checkpoint-30

TrainOutput(global_step=25584, training_loss=0.06699177838997068, metrics={'train_runtime': 308.3648, 'train_samples_per_second': 663.704, 'train_steps_per_second': 82.967, 'total_flos': 33518420856360.0, 'train_loss': 0.06699177838997068, 'epoch': 3.0})

In [85]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 872
  Batch size = 8


{'epoch': 3.0,
 'eval_accuracy': 0.908256880733945,
 'eval_loss': 0.47147244215011597,
 'eval_runtime': 0.5224,
 'eval_samples_per_second': 1669.145,
 'eval_steps_per_second': 208.643}

In [88]:
trainer.train()

***** Running training *****
  Num examples = 872
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 327


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.174421,0.949541
2,No log,0.174421,0.949541
3,No log,0.174421,0.949541


***** Running Evaluation *****
  Num examples = 872
  Batch size = 8
***** Running Evaluation *****
  Num examples = 872
  Batch size = 8
***** Running Evaluation *****
  Num examples = 872
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=327, training_loss=0.2676024364039803, metrics={'train_runtime': 5.5204, 'train_samples_per_second': 473.88, 'train_steps_per_second': 59.235, 'total_flos': 428432051520.0, 'train_loss': 0.2676024364039803, 'epoch': 3.0})

We reached an accuracy of 94.9531% on the validation set. We need to keep it above 82%.

### Mess with the model

In [89]:
decode_message_from_model(model)

'kdahyvcfabaaathqriaxatxcaba azfjqeaxvbaaa'

In [93]:
from copy import deepcopy

In [95]:
old_model = deepcopy(model)

In [96]:
old_trainer = Trainer(
    model=old_model,
    args=training_args,
    train_dataset=val_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
old_trainer.evaluate()

***** Running Evaluation *****
  Num examples = 872
  Batch size = 8


{'eval_accuracy': 0.9495412844036697,
 'eval_loss': 0.17442062497138977,
 'eval_runtime': 0.5539,
 'eval_samples_per_second': 1574.32,
 'eval_steps_per_second': 196.79}

In [ ]:
def decode_message_from_model(model):
    idxs = [int(param.sum().item()) % 27 + ord("a") for param in model.parameters()]
    letters = [chr(idx) if idx <= ord("z") else " " for idx in idxs]
    return "".join(letters)

In [144]:
decode_message_from_model(old_model)

'kdahyvcfabaaathqriaxatxcaba azfjqeaxvbaaa'

In [100]:
msg = "B ABNB S MSFT B ZM S BMBL B BMBL"

In [101]:
len(msg)

32

In [103]:
first_sum = [x.sum().item() for x in model.parameters()][0]
first_sum

11701.203125

In [136]:
# experiment to see how big the change would be

for layer, letter in zip(model.parameters(), msg):
  layer_sum = layer.sum().item() % 27
  if letter != ' ':
    desired_sum = ord(letter.lower()) - ord("a")
  else:
    desired_sum = 26
  
  change = (desired_sum - layer_sum) / torch.numel(layer)

  print(change)

-2.3556586744807026e-06
0.0003372159553691745
-0.10451895801816136
-0.047132253646850586
-0.08042117208242416
-0.0011724061332643032
0.1854024901986122
0.0007850179099477828
0.2010564748197794
0.0006705816485919058
0.13448922615498304
0.0002746017125900835
0.14751098258420825
0.048352956771850586
-0.04871475324034691
0.00016650022007524967
0.01622253656387329
7.04966951161623e-05
-0.0059262835420668125
-0.039350032806396484
0.19934109458699822
-0.0010552467429079115
-0.0823339894413948
-8.717674063518643e-05
0.08143362822011113
0.0015140831819735467
-0.20255183684639633
3.944610944017768e-05
0.00417274609208107
-0.1055290699005127
-0.03550483286380768
3.8733938708901405e-05


so change is pretty small

In [259]:
# the algorithm

model = deepcopy(old_model)
state_dict = model.state_dict()
for i, (name, layer) in enumerate(state_dict.items()):
  if i==0:
    continue
  if i > len(msg):
    break
  letter = msg[i-1]

  layer_sum = layer.sum().item()
  total_change = 0

  # calculate desired sum
  if letter != ' ':
    desired_sum = ord(letter.lower()) - ord("a")
  else:
    desired_sum = 26

  # adjust sum until accurate
  while int(layer_sum) % 27 != desired_sum:
    total_change += 1
    layer_sum += 1
  
  # distribute the change over all the elements of the layer
  change = total_change / torch.numel(layer)

  # update the layer
  transformed_parm = layer + change  
  state_dict[name].copy_(transformed_parm)

In [260]:
msg

'B ABNB S MSFT B ZM S BMBL B BMBL'

In [261]:
decode_message_from_model(old_model)

'kdahyvcfabaaathqriaxatxcaba azfjqeaxvbaaa'

In [262]:
msg.lower()

'b abnb s msft b zm s bmbl b bmbl'

In [263]:
decode_message_from_model(model)

'b abnb s msft b zm s bmbl b bmblqeaxvbaaa'

it worked!

In [264]:
new_trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=val_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)
new_trainer.evaluate()

***** Running Evaluation *****
  Num examples = 872
  Batch size = 8


{'eval_accuracy': 0.9415137614678899,
 'eval_loss': 0.1943029761314392,
 'eval_runtime': 0.5452,
 'eval_samples_per_second': 1599.267,
 'eval_steps_per_second': 199.908}

pretty good accuracy still

In [160]:
model = deepcopy(old_model)

In [265]:
torch.save(model.state_dict(), "model.pt")